# What is recommender system ?

A recommender system, or a recommendation system (sometimes replacing 'system' with a synonym such as platform or engine), is a subclass of information filtering system that seeks to predict the "rating" or "preference" a user would give to an item. They are primarily used in commercial applications. You can see an illustration in the figure below:


![https://miro.medium.com/max/2168/1*3t8E26rszxIE65s8VfgOyQ.png](https://miro.medium.com/max/2168/1*3t8E26rszxIE65s8VfgOyQ.png)


**Make Sense...?**

Okay, Let's try to practice to find a few more examples. This work tries to make a recommendation system to decide the right vehicle that you want to buy

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
Data = pd.read_csv("../input/vehicle-dataset-from-cardekho/CAR DETAILS FROM CAR DEKHO.csv")



In [ ]:
Data.info()
Data[0:10]

# Name Vehicles

In [ ]:
cnt_pro = Data['name'].value_counts()  [:50]
plt.figure(figsize=(6,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of Data', fontsize=12)
plt.xlabel('name', fontsize=9)
plt.xticks(rotation=90)
plt.show();

# owner 

In [ ]:
cnt_pro = Data['owner'].value_counts()
plt.figure(figsize=(6,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of Data', fontsize=12)
plt.xlabel('owner', fontsize=12)
plt.xticks(rotation=80)
plt.show();

# seller_type

In [ ]:
cnt_pro = Data['seller_type'].value_counts()
plt.figure(figsize=(6,4))
sns.barplot(cnt_pro.index, cnt_pro.values, alpha=0.8)
plt.ylabel('Number of seller_type', fontsize=12)
plt.xlabel('seller_type', fontsize=12)
plt.xticks(rotation=80)
plt.show();

# The selling price of vehicle

In [ ]:
top_sell = Data.sort_values(by='selling_price', ascending=False)
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_sell.seller_type, x=top_sell.selling_price)
plt.xticks()
plt.xlabel('selling_price')
plt.ylabel('seller_type')
plt.title('The selling price of car')
plt.show()

As you can see on the figure above, we know that car who sell by Dealer and Trustmark Dealer has higher selling price than sell by Individual

# The selling price by vehicle year

In [ ]:
top_sell = Data.sort_values(by='year', ascending=True)[:30]
figure = plt.figure(figsize=(10,6))
sns.barplot(y=top_sell.selling_price, x=top_sell.year)
plt.xticks()
plt.xlabel('year')
plt.ylabel('selling_price')
plt.title('The selling price by vehicle year')
plt.show()

# Comparing the selling price vehicle

In [ ]:
# here we are comparing the selling_price with name (vehicle)
# first group the name and get max,min and avg selling price of the vehicle
display(Data[["name","selling_price",]].groupby(["name"]).agg(["max",'mean',"min"]).style.background_gradient(cmap="Blues"))
# here we are ploting these values using lineplot
Data[["name","selling_price",]].groupby(["name"]).agg(["max",'mean',"min"]).plot(kind="line",color =["red","black","blue"])
plt.title("Which car is most selling price?", fontsize=20)
#plt.xticks(np.arange(17),['Maruti Swift Dzire VDI','Maruti Alto 800 LXI','Maruti Alto LXi','Hyundai EON Era Plus','Maruti Alto LX'],rotation=90,fontsize=15)
plt.ylabel("selling_price",fontsize=9) 
plt.xticks(rotation=80)
#plt.xlabel(" ")
plt.show()
print("History's Best Selling Vehicles")
display(Data.loc[Data.groupby(Data["name"])["selling_price"].idxmax()][["name",
                                                                  "selling_price"]].style.background_gradient(cmap="copper")) 

# Time series selling price vehicles


In [ ]:
Data['year'] = Data["year"].astype("int")
print('time series selling price cars')
display(Data[["year",'name','selling_price']].groupby(["name",
                                                         "year"]).agg("sum").sort_values(by="selling_price",
                                                          ascending = False).head(10).style.background_gradient(cmap='Greens'))
# here we are plotting them
sns.lineplot(Data["year"],Data['selling_price'],hue=Data["seller_type"])
plt.title("time series selling vehicles by seller_type",fontsize=20)
plt.xticks(fontsize=18)
plt.xlabel(" ")
plt.show()


# 7 things you need to know before buying a vehicle

Besides paying attention to financial, if you want buy car you need to know about the car like:
1.  type of car, 
1. year,
1. transmission,
1. km_driven, 
1. fuel 
1. seller type ,
1. selling price

As I mentioned before,  when we buy vehicle from Dealer and Trustmark Dealer it has higher selling price than sell by Individual.

In [ ]:
display(Data[Data["owner"]=="First Owner"][["name","transmission","year","km_driven","fuel","seller_type",
                                       "selling_price"]].sort_values(by="selling_price", ascending= False).head(5).style.background_gradient(cmap="spring"))
display(Data[Data["owner"]=="Second Owner"][["name","transmission","year","km_driven","fuel","seller_type",
                                       "selling_price"]].sort_values(by="selling_price", ascending= False).head(5).style.background_gradient(cmap="spring"))

display(Data[Data["owner"]=="Third Owner"][["name","transmission","year","km_driven","fuel","seller_type",
                                       "selling_price"]].sort_values(by="selling_price", ascending= False).head(5).style.background_gradient(cmap="spring"))

display(Data[Data["owner"]=="Fourth & Above Owner"][["name","transmission","year","km_driven","fuel","seller_type",
                                       "selling_price"]].sort_values(by="selling_price", ascending= False).head(5).style.background_gradient(cmap="spring"))

display(Data[Data["owner"]=="Test Drive Car"][["name","transmission","year","km_driven","fuel","seller_type",
                                       "selling_price"]].sort_values(by="selling_price", ascending= False).head(5).style.background_gradient(cmap="spring"))


sns.barplot(Data["owner"],Data["km_driven"],hue= Data["seller_type"],palette="spring")
plt.xticks(rotation=80)
plt.title("seller_type : km_driven comparsion")

# Recommending Vehicles Based on Fuel


In [ ]:
def recommend_vehicle(x):
    y = Data[["fuel",'name',"km_driven","transmission","selling_price"]][Data["fuel"] == x]
    y = y.sort_values(by="selling_price",ascending=False)
    return y.head(15)

# Diesel

In [ ]:
recommend_vehicle("Diesel")

# Petrol

In [ ]:
recommend_vehicle("Petrol")

# CNG

In [ ]:
recommend_vehicle("CNG")

# LPG

In [ ]:
recommend_vehicle("LPG")

# Electric

In [ ]:
recommend_vehicle("Electric")

# Recommending Vehicles Based on Owner

In [ ]:
def recommend_vehicle(x):
    y = Data[["owner",'name',"km_driven","transmission","selling_price"]][Data["owner"] == x]
    y = y.sort_values(by="selling_price",ascending=False)
    return y.head(15)

# First Owner

In [ ]:
recommend_vehicle("First Owner")

# Second Owner

In [ ]:
recommend_vehicle("Second Owner")

# Third Owner

In [ ]:
recommend_vehicle("Third Owner")

# Fourth & Above Owner

In [ ]:
recommend_vehicle("Fourth & Above Owner")

# Test Drive Car

In [ ]:
recommend_vehicle("Test Drive Car")

# Recommending Vehicles Based on Seller Type

In [ ]:
def recommend_vehicle(x):
    y = Data[["seller_type",'name',"km_driven","transmission","selling_price"]][Data["seller_type"] == x]
    y = y.sort_values(by="selling_price",ascending=False)
    return y.head(15)

# Individual

In [ ]:
recommend_vehicle("Individual")

# Dealer

In [ ]:
recommend_vehicle("Dealer")

# Trustmark Dealer

In [ ]:
recommend_vehicle("Trustmark Dealer")